## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание входных данных:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Планируется предобработка и анализ представленных данных с последующим ответом на следующие вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

## Открытие файлов с данными и изучение общей информации. 

In [14]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [15]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [16]:
df[df['days_employed']>0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


### Вывод

Обнаружены пропущенные значения в столбцах  days_employed и total_income, причем наблюдается некая зависимость: каждому пропущенному значению столбца days_employed соответсвует пропущенное значение total_income.

Как уже говорилось в легенде к проекту имеются аномальные данные (артефакты) по столбцу days_employed. Наблюдаются как отрицательные, так и положительные значения трудового стажа, причем положительный трудовой стаж присвоен лишь людям с типом занятости ПЕНСИОНЕР. Причиной появления отрицательного стажа могла стать техническая ошибка при расчете: из меньшей даты вычили большую. Так же не ясны причины аномальных данных по стажу в категории ПЕНСИОНЕР, так если данные предствалены в днях, то средний стаж по данной категории 1000 лет. Как вариант - некоррекны единицы измерения. Так как столбец days_employed наименее репрезентативный, то он не был использован при анализе.

Касательно столбца total_income, то данные по доходу в категории ПЕНСИОНЕР так же на мой взгляд не репрезентативны, средней значение около 118 т.р., что не реально для текущих реалий.

## Предобработка данных

### Обработка пропусков

Удаление столбца days_employed - данные не репрезентативны

In [17]:
df = df.drop('days_employed', axis = 'columns')
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...
21520,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Поиск медианных значений для заполнения пропущенных значений по доходу

In [18]:
df[df['total_income'].notna()].groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Определение категорий по которым пропущены значения

In [19]:
df[df['total_income'].isnull()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Ранее были найдены медианы, создал маску замены и заменил nan на соответсвующие значения

In [20]:
mapping = {
    'сотрудник':142594.396847,
    'компаньон':172357.950966,
    'пенсионер':118514.486412,
    'госслужащий':150447.935283    
}
mask = df['total_income'].astype('str').isin(['NaN','nan'])
df.loc[mask, 'total_income'] = df.loc[mask, 'income_type'].map(mapping)

Удалена запись с nan по доходу в категории предприниматель, так как 1 запись не приведет к существенному искажению данных, а заполнение NaN медианными значениями по категории не вижу смысла, так как представлена категория лишь 1 записью

In [21]:
df.dropna(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 11 columns):
children            21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


В сводке статистик указано, что минимальный возраст 0 лет. Фильтрация по условию, где возраст 0

In [22]:
df = df.reset_index(drop = True)
df[df['dob_years'] == 0]['income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

Поиск средних значений для заполнения 0 возраста

In [23]:
df[df['dob_years'] != 0].groupby('income_type')['dob_years'].mean()

income_type
безработный        38.000000
в декрете          39.000000
госслужащий        40.804542
компаньон          39.854294
пенсионер          59.370959
предприниматель    27.000000
сотрудник          40.018980
студент            22.000000
Name: dob_years, dtype: float64

Создание маски замены и сама замена значений с 0

In [24]:
mapping = {'сотрудник':40,
          'пенсионер':59,
          'компаньон':40,
          'госслужащий':40
          }
mask = df['dob_years'].astype('int').isin([0])
df.loc[mask, 'dob_years'] = df.loc[mask, 'income_type'].map(mapping)

Определение уникальных значений по гендерному признаку

In [25]:
df['gender'].value_counts()

F      14236
M       7287
XNA        1
Name: gender, dtype: int64

Поиск индекса пропущенной строки 

In [26]:
df[df['gender'] == 'XNA']

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10700,0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Удаление пропущенной строки 

In [27]:
df = df.drop([10700]).reset_index(drop = True)

Некоторые значения столбца кол-во детей отражены отрицательными, столбец взят по модолю

In [28]:
df['children'] = df['children'].abs()
df.describe()

,children,dob_years,education_id,family_status_id,debt,total_income
count,21523.000000,21523.000000,21523.000000,21523.000000,21523.000000,2.152300e+04
mean,0.543326,43.498955,0.817219,0.972587,0.080890,1.652080e+05
std,1.379930,12.228630,0.548076,1.420374,0.272673,9.802144e+04
min,0.000000,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,34.000000,1.000000,0.000000,0.000000,1.077938e+05
50%,0.000000,43.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,53.000000,1.000000,1.000000,0.000000,1.955403e+05
max,20.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


#### Вывод

Методом df.info() были обнаружены пропущены данные в столбцах days_employed и total_income. Столбец days_employed был полностью удален, так на мой взгляд данные по отдельным категориям не репрезентативны, так же для ответа на поставленные вопросы не нужен. Пропущенных значений в столбце total_income достаточно много 2174 и простое удаление приведет к существенному искажению данных. Я проранжировал данные по типу занятости и заменил пропуски медианными значениями. Методом df.describe() получил сводку статистик, анализируя которую, обнаружил аномальные данные, а именно нулевой возраст и отрицательное кол-во детей. Кол-во детей привел в соответсвии, взяв значение по модолю. Значений возраста с 0 достаточно много, были заменены средними значениями возраста по типу занятости. Сложно сказать почему возникли пропуски, как один из вариантов данные подтягивались в систему банка из ПФР и ФНС и у тех клиентов, где пропущен стаж и доход, неофициальная занятость.     

### Замена типа данных

Привожу доход к целочисленному значению

In [29]:
df['total_income'] = df['total_income'].astype('int')

Перевожу данные об образовании в нижний регистр

In [30]:
df['education'] = df['education'].str.lower()
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...
21518,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21519,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21520,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21521,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [31]:
df[df['income_type'] == 'пенсионер']['total_income'].mean()

135133.41364107883

#### Вывод

Использован метод df.astype() для выделения целочисленного числа по столбцу доход. Использован df.astype() так как преобразование с численным типом данных. Метод df.str.lower() применен к столбцу education для перевода всех значений к нижнему регистру для удобства фильтрации.

### Обработка дубликатов

Поиск дубликатов

In [32]:
df.duplicated().sum()

71

Удаление дубликатов

In [33]:
df = df.drop_duplicates().reset_index(drop = True)
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...
21447,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21448,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21449,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21450,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


#### Вывод

Использованы два метода df.duplicated для поиска дубликатов и df.drop_duplicates() для их удаления. Задвоение могло произойти по причине смены ФИО, система банка могла определить одного человека как две разные личности.

### Лемматизация

Импортирование модуля

In [34]:
from pymystem3 import Mystem
m = Mystem()

Написание функции для леммитизации и добавление соответствующего столбца в df

In [35]:
def find_lemmas(text):
    lemmas = m.lemmatize(text)
    return lemmas
df['lemmas'] = df['purpose'].apply(find_lemmas)

In [36]:
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...
21447,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]"
21448,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21449,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]"
21450,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"


#### Вывод

Произведена леммитизация с помощью модуля Mystem по столбцу purpose. Основные цели кредитования были разбиты на леммы для последующей категоризации

### Категоризация данных

Выделен словарь по виду образования

In [37]:
education_dict = df[['education','education_id']]
rest_education_dict = education_dict.drop_duplicates().reset_index(drop = True)
rest_education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Выделен словарь по семейному статусу

In [38]:
family_status_dict = df[['family_status', 'family_status_id']]
rest_family_status_dict = family_status_dict.drop_duplicates().reset_index(drop = True)
rest_family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Категоризация по кол-ву детей в семье

In [39]:
def children_group(children):
    if children < 1:
        return 'бездетная семья'
    if children < 3:
        return 'малодетная семья'
    if children < 5:
        return 'среднедетная семья'
    return 'многодетная'
df['children_group'] = df['children'].apply(children_group)

Категоризация по доходу

In [40]:
def income_group(income):
    if income < 31000:
        return 'бедные'
    if income < 61000:
        return 'средний класс'
    if income < 100000:
        return 'состоятельные'
    return 'богатые'
df['income_group'] = df['total_income'].apply(income_group)
df['income_group'].value_counts()

богатые          16989
состоятельные     3613
средний класс      824
бедные              26
Name: income_group, dtype: int64

Категоризации по цели кредитования

In [41]:
def purpose_group(purpose):
    for element in purpose:
        if element == 'автомобиль':
            return 'автомобиль'
        elif element == 'свадьба':
            return 'свадьба'
        elif element == 'образование':
            return 'образование'
        elif element == 'недвижимость' or element == 'жилье':
            return 'недвижимость'
df['purpose_group'] = df['lemmas'].apply(purpose_group)
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,children_group,income_group,purpose_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",малодетная семья,богатые,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",малодетная семья,богатые,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",бездетная семья,богатые,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",среднедетная семья,богатые,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",бездетная семья,богатые,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21447,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",малодетная семья,богатые,недвижимость
21448,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",бездетная семья,богатые,автомобиль
21449,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",малодетная семья,состоятельные,недвижимость
21450,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",среднедетная семья,богатые,автомобиль


Проверка корректности категоризации по цели

In [42]:
df['purpose_group'].value_counts()

недвижимость    10809
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

#### Вывод

Для более наглядного представления данных была произведена категоризация по следующим группам: доход, кол-во детей в семье и основные цели кредитования. Доход был разбит на следующие категории: бедные( доход меньше 31 т/р); средний класс( доход от 31 до 61 т/р); состоятельные( доход от 61 до 100 т/р) и богатые( доход свыше 100 т/р). Кол-во детей в семье было разбито на: бездетные(0 детей), малодетные( 1-2 ребенка), среднедетные(3-4 ребенка) и многодетные(5 и более детей). Были выделены следующие основные цели кредитования: недвижимость, образование, автомобиль, свадьба.  

## Поиск закономерностей

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Построена сводная таблица по кол-ву детей в семье и кол-ву возникновения просрочек по кредиту

In [58]:
df_children_debt = df.groupby('children_group').agg({'debt':['sum', 'count']})
df_children_debt['debt_probability_%'] = ((df_children_debt['debt']['sum'] / df_children_debt['debt']['count']) * 100).round(2)
df_children_debt

debt        debt_probability_%
                     sum  count                   
children_group                                    
бездетная семья     1063  14089               7.54
малодетная семья     639   6907               9.25
многодетная            8     85               9.41
среднедетная семья    31    371               8.36

#### Вывод

Была высчитана вероятность появления просрочки для каждой категории семей с детьми. Получилось, что малодетные и многодетные семьи наименее дисциплинированы из всех представленных групп, вероятность выйти на просрочку для них составило около 9%. Так же следует отметить, что бездетные семьи справляются с погашением кредитов лучше остальных групп. Но в целом хотелось бы сказать, что сильного разброса по выделенным группам семей нет, просрачивают кредиты все с вероятностью в пределах от 7% до 9%. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [63]:
df_family_status_debt = df.groupby('family_status').agg({'debt':['sum', 'count']})
df_family_status_debt['debt_probability_%'] = ((df_family_status_debt['debt']['sum'] / df_family_status_debt['debt']['count']) * 100).round(2)
df_family_status_debt

debt        debt_probability_%
                       sum  count                   
family_status                                       
Не женат / не замужем  274   2810               9.75
в разводе               85   1195               7.11
вдовец / вдова          63    959               6.57
гражданский брак       388   4150               9.35
женат / замужем        931  12338               7.55

#### Вывод

По данным сводной таблицы видно, что семья со статусом "не женат / не замужем" и "гражданский брак" более склонны пропускать платежы по кредиту с вероятностью 9%. Наиболее дисплинированной семьей является семья со статусом "вдовец / вдова", вероятность просрочки данной категории составляет 6,57%. Сильного разброса данных также не наблюдается. 

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [65]:
df_income_debt = df.groupby('income_group').agg({'debt':['sum', 'count']})
df_income_debt['debt_probability_%'] = ((df_income_debt['debt']['sum'] / df_income_debt['debt']['count']) * 100).round(2)
df_income_debt

debt        debt_probability_%
                sum  count                   
income_group                                 
бедные            2     26               7.69
богатые        1387  16989               8.16
состоятельные   304   3613               8.41
средний класс    48    824               5.83

#### Вывод

В разрезе данных категорий, наиболее дисциплинированными клиентами являются средний класс - вероятность просрочки 5,83 %. Большие показатели имеют богатые и состоятельные, верояность просрочки составляет более 8%. 

### Как разные цели кредита влияют на его возврат в срок?

In [66]:
df_purpose_debt = df.groupby('purpose_group').agg({'debt':['sum', 'count']})
df_purpose_debt['debt_probability_%'] = ((df_purpose_debt['debt']['sum'] / df_purpose_debt['debt']['count']) * 100).round(2)
df_purpose_debt

debt        debt_probability_%
               sum  count                   
purpose_group                               
автомобиль     403   4306               9.36
недвижимость   782  10809               7.23
образование    370   4013               9.22
свадьба        186   2324               8.00

#### Вывод

Наиболее часто просрачиваемые кредиты - это кредиты на покупку авто и кредит на образование, вероятность просрочек в данных категория более 9 %. Наименее подверженные категории к прасрочкам - это кредиты на покупку недвижимости, вероятность просрочки составляет около 7 %.

## Общий вывод

Первое на хотел бы обратить внимание, так это на корректность данных. На мой взгляд доходы в некоторых категриях по типу занятости весьма завышены. Как пример это средняя запрплата пенсинера 135 т/р. Так же существую артефакты - явно не достоверные данные по трудовому стажу. Однако если отбросить данные замечания, то по данным можно сделать следующий вывод:  

 
1) Наиболее высокий скоринговый бал присвоится заемщику в семейном статусе "вдовец/вдова", без детей со средним заработком до 61 т/р приобретающий недвижимость
2) Наиболее низкий скоринговый бал присвоится заемщику в семейном статусе "не женат/ не замужем", многодетной семьи со средним заработком до 100 т/р приобритающий автомобиль или берущий кредит на цели образования.

В целом картина соответствует действительности. Недвижимость является приоритетным направлением кредитования для среднего класса с доходом до 61 т/р, данная категории семей наиболее ответсвенно относится к своей кредитной истории, так как кредитование один из основных источников финансирования.